# Perform Hyperspectral Analysis and Filtering in Cahmber Window Images

This notebook will explore what types of analyses and filtering can be done with the acquired hyperspectral images. The loaded images will be pre-filtered with a mean filter of kernel size 7. This is done by the file auto_filter.py and should be done beforehand in order to expedite the analysis presented here.

In [1]:
%matplotlib auto
import os
import spectral
import spectral.io.envi as envi
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import pickle
#%%
def save_obj(obj, name ):
    input('ARE YOU SURE YOU WANT TO SAVE THIS OBJECT...')
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def load_hyper_image(filename):
    hdr_name = filename+'.hdr'
    lib = spectral.envi.read_envi_header(hdr_name)
    img = envi.open(hdr_name, filename)
    return lib, img

def mean_spectrum(roi):
    mean_spec = np.mean(roi,axis=1)
    mean_spec = np.mean(mean_spec,axis=0)
    return mean_spec

def get_range(x,y,xlims):
    y = y[x>=xlims[0]]
    x = x[x>=xlims[0]]
    y = y[x<=xlims[1]]
    x = x[x<=xlims[1]]
    return x, y

def get_norm_spec(filename,roi1,roi2):
    lib, img = load_hyper_image(filename)
    roi = img[roi1[0]:roi1[1],roi1[2]:roi1[3],:]
    norm = img[roi2[0]:roi2[1],roi2[2]:roi2[3],:]
    lamb = np.array(lib['wavelength']).astype(float)
    norm_spec = mean_spectrum(roi)/mean_spectrum(norm)
    return lamb,norm_spec

def find_refl(refl,lamb,val):
    diff = np.abs(lamb-val)
    return refl[np.argmin(diff)]

def find_band(lamb,wav):
    diff = np.abs(lamb-wav)
    return np.argmin(diff)

def avg_filter(img,lamb,win):
    def get_kernel(idxx,idxy,win):
        idx_range = np.arange(win)
        shiftx = int(idxx-int(win/2))
        shifty = int(idxy-int(win/2))
        idxx_new = idx_range + shiftx
        idxy_new = idx_range + shifty
        return idxx_new, idxy_new
    mean_kernel = np.zeros([win*win,len(lamb)])
    img2 = img
    for i in range(img.shape[0]-int(win/2)*2):
        for j in range(img.shape[1]-int(win/2)*2):
            idx1 = i + int(win/2)
            idx2 = j + int(win/2)
            rx, ry = get_kernel(idx1,idx2, win)
            count = 0
            for ii in range(len(rx)):
                for jj in range(len(ry)):
                    mean_kernel[count,:] = img_all[rx[ii],ry[jj],:]
                    count = count + 1
            img2[idx1,idx2,:] = np.mean(mean_kernel,axis=0)
        print(i)
        if i == 2000:
            break
    return img2

Using matplotlib backend: Qt5Agg


# Plot the Filtered Image at Certain Wavelengths 

Create a funciton to transform the wavelength ratios to saturation based on ratiometric analysis done in the Notebook absorptivity_rat.ipynb

In [52]:
def get_sat(ratio):
    sat = -10.376376218161289 + 18.27018052*ratio
    return sat

Apply the function to the original images and also plot the wavelength ratios

## Load the Cropped Images
Here, the images are cropped to a region of interest for analysis. This also allows to work with smaller images which expedites the analysis.

In [ ]:
# img1_c = img1[755:1274,116:525,:]
# img2_c = img2[950:1478,250:680,:]
# img3_c = img3[763:1333,20:491,:]

os.chdir('C:\\Users\\Alfredo\\Documents\\University\\FCE\\hyperspectral\\hypoxia\\H002')
img1_c = load_obj('bl')
img2_c = load_obj('hyp2')
img4_c = load_obj('rs4')
img3_c = load_obj('rs2')
lamb = load_obj('lamb')

# Define the colors based on the wavelengths
red = find_band(lamb,620)
green = find_band(lamb,495)
blue = find_band(lamb,450)

img_list = [img1_c,img2_c,img3_c,img4_c]

# Plot the images
# spectral.imshow(img1_c,(red,green,blue),stretch=((0.02, 0.98), (0.02, 0.98), (0.02, 0.98)))
# spectral.imshow(img2_c,(red,green,blue),stretch=((0.02, 0.98), (0.02, 0.98), (0.02, 0.98)))
# spectral.imshow(img3_c,(red,green,blue),stretch=((0.02, 0.98), (0.02, 0.98), (0.02, 0.98)))

Use the image ratios to find a good thresholding level to segment the vessels and tissue background from the image

In [98]:
%matplotlib auto
import matplotlib.cm as cm
num = 576
denom = 486
ratio_list = []
for i in range(len(img_list)):
    plt.subplot(1,len(img_list),1+i)
    ratio1 = img_list[i][:,:,find_band(lamb,num)]/img_list[i][:,:,find_band(lamb,denom)]
    #ratio2 = cv2.medianBlur(ratio2,15)
    plt.imshow(1-ratio1[:,:], cmap=cm.hot, vmin=0, vmax=1)
    cbar = plt.colorbar()
    cbar.set_label('Wavelength Ratio', rotation=270)
    ratio_list.append(ratio1)

Using matplotlib backend: Qt5Agg


Threshold the images created with the wavelength ratios to obtain the background and take the average spectra for the background region

In [103]:
def get_avg_bkg(img, mask):
# mask corresponds to the vessel mask. (i.e. vessels=1, tissue=0)
    img1_t = np.zeros(img.shape)
    img1_v = np.zeros(img.shape)
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if ~(mask[i,j]==True):
                img1_t[i,j,:] = img[i,j,:]
            else:
                img1_v[i,j,:] = img[i,j,:]
    t_mean_1 = img1_t.mean(axis=1).mean(axis=0)
    v_mean_1 = img1_v.mean(axis=1).mean(axis=0)
    return t_mean_1, v_mean_1

t_list = []
v_list = []
for i in range(len(ratio_list)):
    thresh_1 = (1-ratio_list[i][:,:]>0.3)
    plt.imshow(thresh_1)
    t1,v1 = get_avg_bkg(img_list[i],thresh_1)
    t_list.append(t1)
    v_list.append(v1)


C:\Users\Alfredo\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


Plot the average spectra in the vessels of the segmented image

In [63]:
plt.figure()
for i in range(len(t_list)):
    plt.plot(v_list[i]/t_list[i], LINEWIDTH=0.5)
plt.legend(['BL','HYP2','RS2','RS4'])

Find the wavelength ratio that best correlates with the hypoxia state by finding the ratio with the largest difference between the oxygenated and deoxygenated spectra

In [ ]:
# Find the wavelegth ratio that gives the largest difference between the two spectra
n1 = v_list[0]/t_list[0]
n2 = v_list[1]/t_list[1]
diff_mat = np.zeros([len(n1),len(n1)])
for i in range(len(n1)):
    for j in range(len(n1)):
        diff_mat[i,j] = np.abs(n1[i]/n1[j] - n2[i]/n2[j])
num,denom = np.unravel_index(np.nanargmax(diff_mat, axis=None), diff_mat.shape)

Normalize the whole image by the average background and attempt to plot the saturation

In [93]:
img_n = []
for i in range(len(img_list)):
    img_n.append(img_list[i]/t_list[i])

def get_sat(ratio):
    #sat = -14.757682020831949 + 13.45583662*ratio
    sat = ratio
    return sat

num = 467.8
denom = 560.26
vmin = 0
vmax = 0.5
ratio0 = 1 - img_n[2][:,:,find_band(lamb,num)]/img_n[2][:,:,find_band(lamb,denom)]
plt_titles = ['Baseline','Hypoxia 2 min','Resucitation 2 min', 'Resucitation 4 min']
fig = plt.figure()
for i in range(len(img_list)):
    plt.subplot(1,len(img_n),1+i)
    ratio1 = img_n[i][:,:,find_band(lamb,num)]/img_n[i][:,:,find_band(lamb,denom)]
    #ratio2 = cv2.medianBlur(ratio2,15)
    plt.imshow(1-ratio1[:,:], cmap=cm.RdYlBu, vmin=ratio0.min(), vmax=ratio0.max())
    cbar.set_label('Wavelength Ratio', rotation=270)
    plt.title(plt_titles[i])
    plt.xlim([0,400])
    plt.ylim([0,600])
    if i == len(img_n)-1:
        cax = fig.add_axes([0.35, .87, 0.35, 0.03])
        cbar = plt.colorbar(orientation='horizontal', cax=cax, ticks=[-4, 0])
        cbar.set_label('Relative Degree of Oxygenation')
        cbar.ax.set_xticklabels(['DeoxyHB', 'OxyHB'])

In [14]:
def find_max(lamb, img, lamb_min, lamb_max):
    band_min = find_band(lamb,lamb_min)
    band_max = find_band(lamb,lamb_max)
    max_bands = np.argmax(img2[:,:,band_min:band_max],axis=2)
    return max_bands
wav11 = img1_n[:,:,find_band(lamb,555)]
wav2 = np.log(1/img1_n[:,:,find_band(lamb,574)])
wav3 = np.log(1/img1_n[:,:,find_band(lamb,610)])
plt.subplot(2,3,1)
plt.imshow(wav11, cmap=cm.inferno,vmin=wav11.min(), vmax=wav11.max())
plt.subplot(2,3,2)
plt.imshow(wav2, cmap=cm.inferno, vmin=wav1.min(), vmax=1)
plt.subplot(2,3,3)
plt.imshow(wav3, cmap=cm.inferno, vmin=wav1.min(), vmax=1)


wav1 = img2_n[:,:,find_band(lamb,555)]
wav2 = np.log(1/img2_n[:,:,find_band(lamb,574)])
wav3 = np.log(1/img2_n[:,:,find_band(lamb,610)])
plt.subplot(2,3,4)
plt.imshow(wav1, cmap=cm.inferno,vmin=wav11.min(), vmax=wav11.max())
plt.subplot(2,3,5)
plt.imshow(wav2, cmap=cm.inferno, vmin=wav1.min(), vmax=1)
plt.subplot(2,3,6)
plt.imshow(wav3, cmap=cm.inferno, vmin=wav1.min(), vmax=1)